<a href="https://colab.research.google.com/github/23sabriomar/deep_learning/blob/main/dataIntgerationFromWebSites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests --upgrade


     |████████████████████████████████| 62 kB 446 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import requests


Nous pouvons utiliser requests.get pour télécharger une page . Ici, nous devons également définir des en-têtes dans cette fonction car la page Web de Google Scholar nécessite une connexion

In [3]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
url = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=object+detection+in+aerial+image+&btnG=&oq=ob'
response=requests.get(url,headers=headers)


L'objet de réponse contient le résultat des requêtes, le code d'état et d'autres informations. Nous pouvons accéder au contenu de la page Web en utilisant response.text.

***Analyser le code source du code HTML à l'aide d'une belle soupe***

Nous utiliserons la bibliothèque Python Beautiful Soup pour analyser le code source HTML de la page Web téléchargée dans la section précédente. Installons et importons la bibliothèque Beautiful Soup

In [4]:
!pip install beautifulsoup4


Nous pouvons utiliser la BeautifulSoup classe pour analyser un document HTML.

In [5]:
from bs4 import BeautifulSoup


In [ ]:
doc = BeautifulSoup(response.content,'html.parser')

Une fois analysé, nous pouvons utiliser doc pour extraire des informations de la page Web. Maintenant, créons la fonction d'aide réutilisable get_paperinfo qui peut télécharger une page Web et créer un beau document de soupe pour une URL donnée.

In [ ]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):

  #download the page
  response=requests.get(paper_url,headers=headers)

  # check successful response
  if response.status_code != 200:
    print('Status code:', response.status_code)
    raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
  paper_doc = BeautifulSoup(response.text,'html.parser')

  return paper_doc


**Extraction de balises**

Pour extraire les balises, nous allons créer une get_tags fonction qui renverra les balises pour le titre de l'article, la citation, l'URL et l'auteur.

In [ ]:
# this function for the extracting information of the tags
def get_tags(doc):
  paper_tag = doc.select('[data-lid]')
  cite_tag = doc.select('[title=Cite] + a')
  link_tag = doc.find_all('h3',{"class" : "gs_rt"})
  author_tag = doc.find_all("div", {"class": "gs_a"})

  return paper_tag,cite_tag,link_tag,author_tag

**Pour extraire le titre de l' article de recherche** 

nous allons définir la fonction d' aide get_papertitle qui utilisera la liste des balises papier précédemment extraites .

In [ ]:
# it will return the title of the paper
def get_papertitle(paper_tag):
  
  paper_names = []
  
  for tag in paper_tag:
    paper_names.append(tag.select('h3')[0].get_text())

  return paper_names

**Extraction du nombre de citations de l'article**

Maintenant, la get_citecount fonction d'assistance est définie pour extraire les informations du nombre de citations de l'article à l'aide de l'extrait ci-dessus cite_tag . Si l'article n'a pas de citation, le compte sera égal à zéro.

In [ ]:
# it will return the number of citation of the paper
def get_citecount(cite_tag):
  cite_count = []
  for i in cite_tag:
    cite = i.text
    if i is None or cite is None:  # if paper has no citatation then consider 0
      cite_count.append(0)
    else:
      tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
      if tmp is None :
        cite_count.append(0)
      else :
        cite_count.append(int(tmp.group()))

  return cite_count

**Extraction de l'URL du document**

La fonction d'assistance get_link renvoie l'URL de l'article.

In [ ]:
# function for the getting link information
def get_link(link_tag):

  links = []

  for i in range(len(link_tag)) :
    links.append(link_tag[i].a['href']) 

  return links 

**Extraction du nom des auteurs, de l'année et des informations de publication**

Pour extraire le nom des auteurs, l'année de publication de l'article et les informations de la conférence/journal de l'article dans lequel l'article a été publié, nous définirons la fonction get_author_year_publi_info.

In [ ]:
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
  years = []
  publication = []
  authors = []
  for i in range(len(authors_tag)):
      authortag_text = (authors_tag[i].text).split()
      year = int(re.search(r'\d+', authors_tag[i].text).group())
      years.append(year)
      publication.append(authortag_text[-1])
      author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
      authors.append(author)
  
  return years , publication, authors

**Compilez les données et créez un fichier CSV à l'aide de pandas**

Maintenant, nous avons extrait toutes les informations concernant les documents de recherche de Google Scholar. Nous allons stocker ces informations sous forme de dictionnaire.

In [ ]:
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository
def add_in_paper_repo(papername,year,author,cite,publi,link):
  paper_repos_dict['Paper Title'].extend(papername)
  paper_repos_dict['Year'].extend(year)
  paper_repos_dict['Author'].extend(author)
  paper_repos_dict['Citation'].extend(cite)
  paper_repos_dict['Publication'].extend(publi)
  paper_repos_dict['Url of paper'].extend(link)

  return pd.DataFrame(paper_repos_dict)

Dans le message ci-dessus, nous extrairons des informations de la page Web unique. Si nous voulons extraire des informations de plusieurs pages, nous parcourrons chaque page de Google Scholar. Ici, nous grattons les 10 pages de données au total. Nous pouvons gratter plus. Pour cela, nous devons changer le nombre dans la plage.

In [ ]:
for i in range (0,110,10):

  # get url for the each page
  url = "https://scholar.google.com/scholar?start={}&q=object+detection+in+aerial+image+&hl=en&as_sdt=0,5".format(i)

  # function for the get content of each page
  doc = get_paperinfo(url)

  # function for the collecting tags
  paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
  papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
  year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
  cite = get_citecount(cite_tag)

  # url of the paper
  link = get_link(link_tag)

  # add in paper repo dict
  final = add_in_paper_repo(papername,year,author,cite,publication,link)
  
  # use sleep to avoid status code 429
  sleep(30)